In [ ]:
import os
import pandas as pd
import time
import glob
from IPython.display import clear_output


from pyspark.sql import SparkSession
import pyspark.sql.functions as sql_f
from pyspark.sql.types import *
from pyspark.sql.functions import to_date, datediff, floor, col, avg, substring, when, length, lpad
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import unix_timestamp

spark = SparkSession.builder.getOrCreate()

In [ ]:
#Function to measure execution time...
def time_execution(task_name, func):
    start_time = time.time()
    result = func()
    end_time = time.time()
    duration = end_time - start_time
    print(f"{task_name} executed in {duration:.2f} seconds")
    return result, duration


## 1) Loading the data

In [ ]:
#Synthea Batch Generator with Concatenation of csv files

#Synthea Configuration 
total_patients = 200  
batch_size = 20
state = "Massachusetts"  
age_range = "30-85"  
base_seed = 12345  
output_dir = "./output" 

#Initial setup to ensure java is installed as well as the "synthea-with-dependencies.jar" file needed to create the patient records
def setup_environment():
    """Install Java and download Synthea if not already present"""
    if not os.path.exists("/usr/bin/java"):
        print("Installing Java...")
        !sudo apt-get update -qq > /dev/null
        !sudo apt-get install -y openjdk-11-jdk-headless > /dev/null
        clear_output()
        print("Java installed")
    
    if not os.path.exists("synthea-with-dependencies.jar"):
        print("Downloading Synthea...")
        !wget -q https://github.com/synthetichealth/synthea/releases/download/master-branch-latest/synthea-with-dependencies.jar
        clear_output()
        print("Synthea downloaded")
    
    os.makedirs(output_dir, exist_ok=True)

#Batch generation of patients to avoid errors due to kernel reconnecting...
def generate_batch(batch_num, patients_in_batch, current_seed):
    """Generate one batch of synthetic patients"""
    try:
        print(f"Batch {batch_num}: Generating {patients_in_batch} patients (seed: {current_seed})")
        
        !java -jar synthea-with-dependencies.jar \
          -p {patients_in_batch} \
          -s {current_seed} \
          -a "{age_range}" \
          --exporter.baseDirectory "{output_dir}" \
          --exporter.fhir.export=False \
          --exporter.csv.export=True \
          --exporter.csv.folder_per_run=true \
          {state}
        
        return True
    except Exception as e:
        print(f"Error in batch {batch_num}: {str(e)}")
        return False

#Combining csv files based on their filenames
def concatenate_all_csvs():
    """Combine all generated CSV files by their type"""
    csv_files = glob.glob(f"{output_dir}/**/*.csv", recursive=True)
    
    if not csv_files:
        print("No CSV files found to concatenate")
        return None
    
    combined_data = {}
    
    for filepath in csv_files:
        filename = os.path.basename(filepath)
        file_type = filename.split('.')[0]
        
        try:
            df = pd.read_csv(filepath)
            
            if file_type in combined_data:
                combined_data[file_type] = pd.concat([combined_data[file_type], df], ignore_index=True)
            else:
                combined_data[file_type] = df
                
        except Exception as e:
            print(f"Could not process {filename}: {str(e)}")
    
    #Saving combined files
    combined_dir = f"{output_dir}/combined"
    os.makedirs(combined_dir, exist_ok=True)
    
    for file_type, df in combined_data.items():
        output_path = f"{combined_dir}/{file_type}.csv"
        df.to_csv(output_path, index=False)
        print(f"Saved {len(df)} records to {output_path}")
    
    return combined_data

#Calling functions from above:
if __name__ == "__main__":
    setup_environment()

    start_time = time.time()
    completed = 0
    batch_num = 1
    
    print(f"Starting generation of {total_patients} patients in batches of {batch_size}...")
    
    while completed < total_patients:
        current_batch_size = min(batch_size, total_patients - completed)
        current_seed = base_seed + completed
        
        #Retrying logic (3 attempts per batch)
        success = False
        for attempt in range(3):
            if generate_batch(batch_num, current_batch_size, current_seed):
                success = True
                break
            time.sleep(5)
        
        if success:
            completed += current_batch_size
            progress = completed / total_patients * 100
            print(f"Progress: {completed}/{total_patients} ({progress:.1f}%)")
            batch_num += 1
        else:
            print(f"Failed batch {batch_num} after 3 attempts")
            break
    
    #Concatenate results
    print("\nCombining all CSV files...")
    combined_data = concatenate_all_csvs()

    
    elapsed = (time.time() - start_time) / 60
    print(f"\n{'='*40}")
    print(f"COMPLETED IN {elapsed:.1f} MINUTES")
    print(f"Total patients generated: {completed}/{total_patients}")
    
    if combined_data:
        print("\nCOMBINED FILES SUMMARY:")
        for file_type, df in combined_data.items():
            print(f"- {file_type}.csv: {len(df)} records")
    else:
        print("\nNo files were combined")

In [ ]:
#moving combined csv files into HDFS
!hdfs dfs -mkdir -p /synthea_output 2> /dev/null
!hdfs dfs -put -f ./output/combined/*.csv /synthea_output/
!rm -rf ./output 2> /dev/null && echo "Local files cleaned up" || echo "Error cleaning local files"

In [ ]:
#uncomment this cell to save files to local downloads:

# !mkdir -p ~/Downloads/synthea_csvs
# !hdfs dfs -get /synthea_output/*.csv ~/Downloads/synthea_csvs/

## 2) Creating the spark dataframes 

In [ ]:
#setting path to HDFS folder
path = '/synthea_output/'


In [ ]:
#Patient files
observations = spark.read.csv(path+"observations.csv", header=True)
patient = spark.read.csv(path+"patients.csv", header=True) 
#Medical files
careplans = spark.read.csv(path+"careplans.csv", header=True)
conditions = spark.read.csv(path+"conditions.csv", header=True)
procedures=spark.read.csv(path+"procedures.csv", header=True)
encounters = spark.read.csv(path+"encounters.csv", header=True)
medications = spark.read.csv(path+"medications.csv", header=True)
#Insurance and hospital files
payer_transitions=spark.read.csv(path+"payer_transitions.csv", header=True)
payers=spark.read.csv(path+"payers.csv", header=True)
providers=spark.read.csv(path+"providers.csv", header=True)
organizations=spark.read.csv(path+"organizations.csv", header=True)

## 3) Cleaning dataframes and renaming variables 

In [ ]:
#Renaming columns
patient = (
    patient.withColumnRenamed("Id", "patient_id")
           .withColumnRenamed("MARITAL", "patient_marital")
           .withColumnRenamed("RACE", "patient_race")
           .withColumnRenamed("ETHNICITY", "patient_ethnicity")
           .withColumnRenamed("GENDER", "patient_gender")
           .withColumnRenamed("ZIP", "patient_zip")
)
encounters = (
    encounters.withColumnRenamed("PATIENT", "patient_id")
              .withColumnRenamed("Id", "encounter_id")
              .withColumnRenamed("DESCRIPTION", "encounter_discription")
              .withColumnRenamed("CODE", "encounter_code")
              .withColumnRenamed("START", "encounter_start")
              .withColumn("encounter_start", to_date("encounter_start"))
              .withColumnRenamed("STOP", "encounter_stop")
              .withColumn("encounter_stop", to_date("encounter_stop"))
              .withColumn("PATIENT COST", col("TOTAL_CLAIM_COST") - col("PAYER_COVERAGE"))
              .withColumnRenamed("PAYER", "payer_id")
              .withColumnRenamed("ORGANIZATION", "organization_id")
              .withColumnRenamed("PROVIDER", "provider_id")
)
careplans = (
    careplans.withColumnRenamed("PATIENT", "patient_id")
             .withColumnRenamed("Id", "careplan_id")
             .withColumnRenamed("ENCOUNTER", "encounter_id")
             .withColumnRenamed("DESCRIPTION", "careplan_descriptions")
             .withColumnRenamed("CODE", "careplan_code")
)
procedures = (
    procedures.withColumnRenamed("PATIENT", "patient_id")
              .withColumnRenamed("ENCOUNTER", "encounter_id")
              .withColumnRenamed("DESCRIPTION", "procedure_descriptions")
              .withColumnRenamed("CODE", "procedure_code")
              .withColumnRenamed("DATE", "procedure_date")
              .withColumnRenamed("BASE_COST", "procedure_cost")
)
conditions = (
    conditions.withColumnRenamed("PATIENT", "patient_id")
              .withColumnRenamed("ENCOUNTER", "encounter_id")
              .withColumnRenamed("DESCRIPTION", "condition_description")
              .withColumnRenamed("CODE", "condition_code")
              .withColumnRenamed("START", "condition_start")
              .withColumnRenamed("END", "condition_end")
)
observations = (
    observations.withColumnRenamed("PATIENT", "patient_id")
                .withColumnRenamed("ENCOUNTER", "encounter_id")
                .withColumnRenamed("DATE", "observation_date")
                .withColumn("observation_date", to_date("observation_date"))
                .withColumn("obs_value", col("VALUE").cast("double"))
                .withColumnRenamed("CODE", "observation_code")
                .withColumnRenamed("DESCRIPTION", "observation_description")
)
medications = (
    medications.withColumnRenamed("START", "medication_start")
               .withColumn("medication_start", to_date("medication_start"))
               .withColumnRenamed("STOP", "medication_stop")
               .withColumn("medication_stop", to_date("medication_stop"))
               .withColumnRenamed("PATIENT", "patient_id")
               .withColumnRenamed("PAYER", "payer_id")
               .withColumnRenamed("ENCOUNTER", "encounter_id")
               .withColumnRenamed("CODE", "medication_code")
               .withColumnRenamed("DESCRIPTION", "medication_description")
)
payer_transitions = (
    payer_transitions.withColumnRenamed("PATIENT", "patient_id")
                     .withColumnRenamed("PAYER", "payer_id")
)
payers = (
    payers.withColumnRenamed("Id", "payer_id")
          .withColumnRenamed("NAME", "payer_name")
          .withColumnRenamed("OWNERSHIP", "payer_ownership")
)
providers = (
    providers.withColumnRenamed("Id", "provider_id")
             .withColumnRenamed("SPECIALITY", "provider_specialty")
)
organizations = (
    organizations.withColumnRenamed("Id", "organization_id")
                 .withColumnRenamed("NAME", "organization_name")
                 .withColumnRenamed("ZIP", "organization_zip")
)
organizations = organizations.withColumn(
    "organization_zip",
    col("organization_zip").cast("string")
)

#adding leading 0's to zip codes to retain their information
organizations = organizations.withColumn(
    "organization_zip",
    when(length(col("organization_zip")) == 4,  
    lpad(col("organization_zip"), 5, "0")
).otherwise(col("organization_zip")))   

In [ ]:
#Merge together dataframes on various id fields that will be used for ML modeling...
encounters = (
    encounters
    .join(payers.select("payer_id", "payer_name", "payer_ownership"), on="payer_id", how="left")
    .join(organizations.select("organization_id", "organization_name", "organization_zip"), on="organization_id", how="left")
    .join(providers.select("provider_id", "provider_specialty"), on="provider_id", how="left")
    .join(procedures.select("encounter_id", "procedure_descriptions", "procedure_code"), on="encounter_id", how="left")
    .join(patient.select("patient_id", "BIRTHDATE", "patient_marital", "patient_race", "patient_ethnicity", "patient_gender", "patient_zip"), on="patient_id", how="left")
    .withColumn("age_at_encounter", floor(datediff(col("encounter_start"), col("BIRTHDATE")) / 365.25))
)


In [29]:
encounters.show(5)

25/05/05 21:33:22 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------+--------------+--------------+---------------------+-------------------+----------------+--------------+----------+-----------------+------------+----------+---------------+--------------------+----------------+------------------+----------------------+--------------+----------+---------------+------------+-----------------+--------------+-----------+----------------+
|          patient_id|        encounter_id|         provider_id|     organization_id|            payer_id|encounter_start|encounter_stop|ENCOUNTERCLASS|encounter_code|encounter_discription|BASE_ENCOUNTER_COST|TOTAL_CLAIM_COST|PAYER_COVERAGE|REASONCODE|REASONDESCRIPTION|PATIENT COST|payer_name|payer_ownership|   organization_name|organization_zip|provider_specialty|procedure_descriptions|procedure_code| BIRTHDATE|patient_marital|patient_race|patient_ethnicity|patient_gender|patient_zip|ag

In [9]:
encounters.columns

['patient_id',
 'encounter_id',
 'provider_id',
 'organization_id',
 'payer_id',
 'encounter_start',
 'encounter_stop',
 'ENCOUNTERCLASS',
 'encounter_code',
 'encounter_discription',
 'BASE_ENCOUNTER_COST',
 'TOTAL_CLAIM_COST',
 'PAYER_COVERAGE',
 'REASONCODE',
 'REASONDESCRIPTION',
 'PATIENT COST',
 'payer_name',
 'payer_ownership',
 'organization_name',
 'organization_zip',
 'provider_specialty',
 'procedure_descriptions',
 'procedure_code',
 'BIRTHDATE',
 'patient_marital',
 'patient_race',
 'patient_ethnicity',
 'patient_gender',
 'patient_zip',
 'age_at_encounter']

['patient_id',
 'encounter_id',
 'provider_id',
 'organization_id',
 'payer_id',
 'encounter_start',
 'encounter_stop',
 'ENCOUNTERCLASS',
 'encounter_code',
 'encounter_discription',
 'BASE_ENCOUNTER_COST',
 'TOTAL_CLAIM_COST',
 'PAYER_COVERAGE',
 'REASONCODE',
 'REASONDESCRIPTION',
 'PATIENT COST',
 'payer_name',
 'payer_ownership',
 'organization_name',
 'organization_zip',
 'provider_specialty',
 'procedure_descriptions',
 'procedure_code',
 'BIRTHDATE',
 'patient_marital',
 'patient_race',
 'patient_ethnicity',
 'patient_gender',
 'patient_zip',
 'age_at_encounter']

## Preparing Data

In [10]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

modeling_df = encounters.select(
    col("PATIENT COST").cast("double").alias("label"),
    col("age_at_encounter").cast("double"),
    col("patient_marital"),
    col("patient_race"),
    col("patient_ethnicity"),
    col("patient_gender"),
    col("ENCOUNTERCLASS"),
    col("payer_ownership"),
    col("payer_name"),
    col("organization_zip"),
    col("organization_name"),
    col("procedure_code"),
    #col("encounter_discription"), # don't use for RF, LR and GBT models
    col("encounter_code"),
    #col("REASONDESCRIPTION"), # don't use for RF, LR and GBT models
).na.drop().filter(col("PATIENT COST") != 0)


# Define categorical and numeric columns
categorical_cols = ['patient_marital', 'patient_race', 'patient_ethnicity', 
                   'patient_gender', 'ENCOUNTERCLASS',
                   'payer_ownership',"payer_name","organization_name", "organization_zip", 'procedure_code',"encounter_code"]
numeric_cols = ['age_at_encounter']

target_col = "label"

# Create feature engineering pipeline stages
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid="keep") 
            for col in categorical_cols]

encoder = OneHotEncoder(
    inputCols=[col+"_index" for col in categorical_cols],
    outputCols=[col+"_encoded" for col in categorical_cols],
    dropLast=True
)

assembler = VectorAssembler(
    inputCols=numeric_cols + [col+"_encoded" for col in categorical_cols],
    outputCol="features"
)

# Cache after feature engineering
feature_pipeline = Pipeline(stages=indexers + [encoder, assembler])
feature_model = feature_pipeline.fit(modeling_df)
feature_df = feature_model.transform(modeling_df).select("features", target_col).cache()

In [28]:
feature_df.count()

45140

In [11]:
#Splitting training and test dataframes nd present the number of partitions of the training dataframe
train_df, test_df = time_execution(
    "Data splitting",
    lambda: feature_df.randomSplit([0.8, 0.2], seed=53)
)[0]
train_df.rdd.getNumPartitions()

Data splitting executed in 0.04 seconds
Data splitting executed in 0.04 seconds


1

1

In [16]:
# Repartition data to distribute it across the cluster
train_df = train_df.repartition(12)

# Optional: Check how many partitions are in the RDD
print(f"Training Data Partitions: {train_df.rdd.getNumPartitions()}")

Training Data Partitions: 12


## Random Forest Model

In [17]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import time

# Common evaluator for regression tasks
reg_evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction")

rf = RandomForestRegressor(
    featuresCol="features",
    labelCol="label",
    numTrees=30,  # Reduced for initial testing
    maxDepth=10,
    subsamplingRate=0.7,
    featureSubsetStrategy='sqrt',
    seed=42
)

rf_model, rf_time = time_execution(
    "Random Forest training",
    lambda: rf.fit(train_df)
)

rf_predictions = rf_model.transform(test_df)


print("Random Forest Regression Results:")
print("RMSE:", reg_evaluator.evaluate(rf_predictions, {reg_evaluator.metricName: "rmse"}))
print("R2:", reg_evaluator.evaluate(rf_predictions, {reg_evaluator.metricName: "r2"}))

Random Forest training executed in 19.26 seconds
Random Forest Regression Results:
RMSE: 1747.8312763425588
R2: 0.7661240686501508


In [27]:
rf_predictions.show(20)

+--------------------+------------------+------------------+
|            features|             label|        prediction|
+--------------------+------------------+------------------+
|(577,[0,1,5,8,10,...|            159.11|1078.3459026211667|
|(577,[0,1,5,8,10,...|            159.11|1078.3459026211667|
|(577,[0,1,5,8,10,...| 411.7700000000002| 992.5397008297834|
|(577,[0,1,5,8,10,...|185.55999999999995| 940.0779554792844|
|(577,[0,1,5,8,10,...|            159.11| 940.0779554792844|
|(577,[0,1,5,8,10,...| 480.5000000000001|1265.2417193356916|
|(577,[0,1,5,8,10,...| 411.7700000000002|1095.5674661387266|
|(577,[0,1,5,8,10,...|185.55999999999995|1043.1057207882272|
|(577,[0,1,5,8,10,...|172.90999999999997|1043.1057207882272|
|(577,[0,1,5,8,10,...|172.90999999999997|1043.1057207882272|
|(577,[0,1,5,8,10,...|172.90999999999997|1043.1057207882272|
|(577,[0,1,5,8,10,...|            159.11|1043.1057207882272|
|(577,[0,1,5,8,10,...|            159.11|1043.1057207882272|
|(577,[0,1,5,8,10,...|36

## Linear Regression Model

In [19]:
### Linear Regression
lr = LinearRegression(
    featuresCol="features",
    labelCol="label",
    regParam=0.01,
    elasticNetParam=0.5
)

lr_model, lr_time = time_execution(
    "Linear Regression training",
    lambda: lr.fit(train_df)
)

lr_predictions = lr_model.transform(test_df)

print("\nLinear Regression Results:")
print("RMSE:", reg_evaluator.evaluate(lr_predictions, {reg_evaluator.metricName: "rmse"}))
print("R2:", reg_evaluator.evaluate(lr_predictions, {reg_evaluator.metricName: "r2"}))


Linear Regression training executed in 6.97 seconds

Linear Regression Results:
RMSE: 1803.6438931701152
R2: 0.7509491041116775


In [26]:
lr_predictions.show(20)

+--------------------+------------------+------------------+
|            features|             label|        prediction|
+--------------------+------------------+------------------+
|(577,[0,1,5,8,10,...|            159.11|-705.2915888145076|
|(577,[0,1,5,8,10,...|            159.11|-705.2915888145076|
|(577,[0,1,5,8,10,...| 411.7700000000002|-662.1490766850343|
|(577,[0,1,5,8,10,...|185.55999999999995|-273.8664675197756|
|(577,[0,1,5,8,10,...|            159.11| 71.27362951600983|
|(577,[0,1,5,8,10,...| 480.5000000000001|  963.179869443896|
|(577,[0,1,5,8,10,...| 411.7700000000002|137.94835713634055|
|(577,[0,1,5,8,10,...|185.55999999999995| 526.2309663015994|
|(577,[0,1,5,8,10,...|172.90999999999997| 828.2285512079118|
|(577,[0,1,5,8,10,...|172.90999999999997| 828.2285512079118|
|(577,[0,1,5,8,10,...|172.90999999999997| 828.2285512079118|
|(577,[0,1,5,8,10,...|            159.11| 871.3710633373851|
|(577,[0,1,5,8,10,...|            159.11| 871.3710633373851|
|(577,[0,1,5,8,10,...|36

## Gradient Boosted Tree Model

In [21]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml import Pipeline

# Gradient Boosted Trees Regressor
gbt = GBTRegressor(
    featuresCol="features",
    labelCol="label",
    maxIter=20,  # Reduced iterations
    maxDepth=10,
    stepSize=0.1,
    subsamplingRate=0.7,
    seed=42
)

gbt_model, gbt_time = time_execution(
    "GBT training",
    lambda: gbt.fit(train_df)
)

gbt_predictions = gbt_model.transform(test_df)

print("\nGBT Regression Results:")
print("RMSE:", reg_evaluator.evaluate(gbt_predictions, {reg_evaluator.metricName: "rmse"}))
print("R2:", reg_evaluator.evaluate(gbt_predictions, {reg_evaluator.metricName: "r2"}))

GBT training executed in 119.60 seconds

GBT Regression Results:



[Stage 850:>                                                        (0 + 1) / 1]



RMSE: 607.5510575748652
R2: 0.9717413064781539


In [25]:
gbt_predictions.show(20)

+--------------------+------------------+------------------+
|            features|             label|        prediction|
+--------------------+------------------+------------------+
|(577,[0,1,5,8,10,...|            159.11| 648.4175865061991|
|(577,[0,1,5,8,10,...|            159.11| 648.4175865061991|
|(577,[0,1,5,8,10,...| 411.7700000000002| 648.4175865061991|
|(577,[0,1,5,8,10,...|185.55999999999995| 234.7496735154522|
|(577,[0,1,5,8,10,...|            159.11| 234.7496735154522|
|(577,[0,1,5,8,10,...| 480.5000000000001|422.49699698298963|
|(577,[0,1,5,8,10,...| 411.7700000000002| 648.4175865061991|
|(577,[0,1,5,8,10,...|185.55999999999995| 234.7496735154522|
|(577,[0,1,5,8,10,...|172.90999999999997| 234.7496735154522|
|(577,[0,1,5,8,10,...|172.90999999999997| 234.7496735154522|
|(577,[0,1,5,8,10,...|172.90999999999997| 234.7496735154522|
|(577,[0,1,5,8,10,...|            159.11| 234.7496735154522|
|(577,[0,1,5,8,10,...|            159.11| 234.7496735154522|
|(577,[0,1,5,8,10,...|36

## Ensemble of ML Models 

In [23]:
from pyspark.ml.regression import RandomForestRegressor, LinearRegression, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from concurrent.futures import ThreadPoolExecutor
from pyspark.sql.functions import monotonically_increasing_id, expr

# Split the training data into 3 parts for the 3 models
rf_df, lr_df, gbt_df = train_df.randomSplit([1.0, 1.0, 1.0], seed=42)

# Define model training functions
def train_rf():
    rf = RandomForestRegressor(featuresCol="features", labelCol="label", numTrees=30, maxDepth=10, seed=42)
    return rf.fit(rf_df)

def train_lr():
    lr = LinearRegression(featuresCol="features", labelCol="label", regParam=0.01, elasticNetParam=0.5)
    return lr.fit(lr_df)

def train_gbt():
    gbt = GBTRegressor(featuresCol="features", labelCol="label", maxIter=10, maxDepth=5, stepSize=0.1, seed=42)
    return gbt.fit(gbt_df)

# Train models in parallel
with ThreadPoolExecutor() as executor:
    rf_future = executor.submit(train_rf)
    lr_future = executor.submit(train_lr)
    gbt_future = executor.submit(train_gbt)

    rf_model = rf_future.result()
    lr_model = lr_future.result()
    gbt_model = gbt_future.result()

# Make predictions and add row indices
rf_pred = rf_model.transform(test_df).select("prediction", "label") \
    .withColumnRenamed("prediction", "rf_pred") \
    .withColumn("row_idx", monotonically_increasing_id())

lr_pred = lr_model.transform(test_df).select("prediction") \
    .withColumnRenamed("prediction", "lr_pred") \
    .withColumn("row_idx", monotonically_increasing_id())

gbt_pred = gbt_model.transform(test_df).select("prediction") \
    .withColumnRenamed("prediction", "gbt_pred") \
    .withColumn("row_idx", monotonically_increasing_id())

# Join predictions on row index
ensemble_df = rf_pred \
    .join(lr_pred, on="row_idx") \
    .join(gbt_pred, on="row_idx")

# Compute average prediction
ensemble_df = ensemble_df.withColumn("ensemble_prediction", expr("(rf_pred + lr_pred + gbt_pred)/3"))

# Evaluate ensemble
reg_evaluator = RegressionEvaluator(labelCol="label", predictionCol="ensemble_prediction")

print("\nEnsemble Results:")
print("RMSE:", reg_evaluator.evaluate(ensemble_df, {reg_evaluator.metricName: "rmse"}))
print("R2:", reg_evaluator.evaluate(ensemble_df, {reg_evaluator.metricName: "r2"}))


25/05/05 21:31:23 WARN DAGScheduler: Broadcasting large task binary with size 1208.4 KiB
25/05/05 21:31:27 WARN DAGScheduler: Broadcasting large task binary with size 1565.9 KiB
25/05/05 21:31:32 WARN DAGScheduler: Broadcasting large task binary with size 2008.1 KiB
25/05/05 21:31:37 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB



Ensemble Results:
RMSE: 998.4577255748717
R2: 0.9236786787133897


In [24]:
ensemble_df.show(20)

+-------+-----------------+------------------+-------------------+-----------------+-------------------+
|row_idx|          rf_pred|             label|            lr_pred|         gbt_pred|ensemble_prediction|
+-------+-----------------+------------------+-------------------+-----------------+-------------------+
|      0|704.9655962776457|            159.11| -606.4865431954435|401.1089130420247| 166.52932204140896|
|      1|704.9655962776457|            159.11| -606.4865431954435|401.1089130420247| 166.52932204140896|
|      2|704.9655962776457| 411.7700000000002| -564.6929316237091|401.1089130420247| 180.46052589865374|
|      3|620.3533124835761|185.55999999999995|-188.55042747810228|401.1089130420247| 277.63726601583284|
|      4|683.7251174627415|            159.11| 145.79846509577067|401.1089130420247| 410.21083186684564|
|      5|620.3533124835761| 480.5000000000001| 1331.6097492300871|401.1089130420247|  784.3573249185625|
|      6|704.9655962776457| 411.7700000000002|  798.073